In [ ]:
import os,sys,inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir) 

from utils.io_utils import IOUtils
from utils.nlp_utils import NLPUtils

Using the default treebank "en_ewt" for language "en".
Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/home/huseyinalecakir/Security/source/PermissionDescriptionFidelity/utils/../../../data/models/en_ewt_models/en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: depparse
With settings: 
{'model_path': '/home/huseyinalecakir/Security/source/PermissionDescriptionFidelity/utils/../../../data/models/en_ewt_models/en_ewt_parser.pt', 'pretrain_path': '/home/huseyinalecakir/Security/source/PermissionDescriptionFidelity/utils/../../../data/models/en_ewt_models/en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}


In [ ]:
DIR_NAME = os.path.abspath('')
IN_PATH = os.path.join(DIR_NAME, "../../../data/play_store_data/record_audio_descriptions.csv")
OUT_PATH = os.path.join(DIR_NAME, "../../../data/play_store_data/app_ids_record_audio_processed.xls")

In [ ]:
import csv, time
def clean_play_store_data(file_path):
    """TODO"""
    number_of_apps = 0
    data = {}
    with open(file_path) as stream:
        reader = csv.reader(stream)
        next(reader)
        start_time = time.time()
        for row in reader:
            number_of_apps += 1
            app_id = row[0]
            text = row[1]
            data[app_id] = []
            for sentence in NLPUtils.sentence_tokenization(text):
                data[app_id].append(sentence)
    return data

In [ ]:
import re
def remove_given_pattern(regex, data):
    updated_data = {}
    for  key in list(data.keys()):
        new_lines = []
        for line in data[key]:
            updated = re.sub(regex, '', line)
            new_lines.append(updated)
        updated_data[key] = new_lines
    return updated_data


In [ ]:
def remove_emoji(data):
    updated_data = {}
    import demoji
    demoji.download_codes()
    for key in list(data.keys()):
        new_lines = []
        for line in data[key]:
            updated = demoji.replace(line, repl="").strip()
            if updated:
                new_lines.append(updated)
        updated_data[key] = new_lines
    return updated_data

In [ ]:
def write_excel(data, outfile, count=1000):
    import xlwt
    header = ["Sentences", "Manually Marked"]
    workbook = xlwt.Workbook()
    sheet = workbook.add_sheet('Sheet')
    sheet.write(0, 0, "Sentences") 
    sheet.write(0, 1, "Manually Marked") 
    row_number = 1
    for idx, app_id in zip(range(count), data):
        sheet.write(row_number, 0, "##{}".format(app_id))
        row_number += 1
        for sentence in data[app_id]:
            sheet.write(row_number, 0, sentence)
            row_number += 1
    workbook.save(outfile)


In [ ]:
regex = r"^[^\w\!\?\\\(\)\[\]\“\‘\"]+"
data = clean_play_store_data(IN_PATH)
data = remove_given_pattern(regex, data)
data = remove_emoji(data)
write_excel(data, OUT_PATH, 1000)